# 1.0 - Limpieza de datos: Notebook de Angie Alfonso
En este notebook voy a trabajar la parte relacionada con la **estructura de la vivienda, su exterior y las calificaciones de calidad**.  
El objetivo es dejar estas variables listas para el análisis y modelado, eliminando o corrigiendo valores nulos y asegurando que los tipos de datos sean los adecuados.


In [28]:
# --- CONFIGURACIÓN INICIAL ---
%load_ext autoreload
%autoreload 2

from housing_price_prediction.utils.paths import data_raw_dir
import pandas as pd
import numpy as np
import seaborn as sns

# Cargar dataset
raw_data_path = data_raw_dir() / "train.csv"
df_data_raw = pd.read_csv(raw_data_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Selección de las columnas a trabajar

Nos enfocaremos en las variables relacionadas con la **estructura**, **exterior** y **calidad** de las viviendas.  
- `OverallQual`, `OverallCond`: calificación general de calidad y condición.  
- `YearBuilt`, `YearRemodAdd`: año de construcción y remodelación.  
- `RoofStyle`, `RoofMatl`: tipo y material del techo.  
- `Exterior1st`, `Exterior2nd`: materiales exteriores principales.  
- `MasVnrType`, `MasVnrArea`: tipo y área del recubrimiento de mampostería.  
- `ExterQual`, `ExterCond`, `HeatingQC`: calidades y condiciones.  
- `Foundation`, `Heating`, `CentralAir`, `Electrical`: estructura base, tipo de calefacción, aire acondicionado y sistema eléctrico.


In [29]:
cols = [
    'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
    'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
    'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond',
    'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical'
]

#Creacion de un sub-dataframe para trabajar con las variables asignadas evitando modificar accidentalmente la data original
df = df_data_raw[cols].copy()

## Exploración de los datos 


Revisión de los tipos de datos iniciales

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OverallQual   1460 non-null   int64  
 1   OverallCond   1460 non-null   int64  
 2   YearBuilt     1460 non-null   int64  
 3   YearRemodAdd  1460 non-null   int64  
 4   RoofStyle     1460 non-null   object 
 5   RoofMatl      1460 non-null   object 
 6   Exterior1st   1460 non-null   object 
 7   Exterior2nd   1460 non-null   object 
 8   MasVnrType    588 non-null    object 
 9   MasVnrArea    1452 non-null   float64
 10  ExterQual     1460 non-null   object 
 11  ExterCond     1460 non-null   object 
 12  Foundation    1460 non-null   object 
 13  Heating       1460 non-null   object 
 14  HeatingQC     1460 non-null   object 
 15  CentralAir    1460 non-null   object 
 16  Electrical    1459 non-null   object 
dtypes: float64(1), int64(4), object(12)
memory usage: 194.0+ KB


## Revisión de valores nulos 


Antes de dividir en variables categóricas y numéricas hago un conteo de nulos para entender qué columnas necesitan más atención.

In [31]:
# Contar cuántos valores nulos hay en cada columna
df.isna().sum()

OverallQual       0
OverallCond       0
YearBuilt         0
YearRemodAdd      0
RoofStyle         0
RoofMatl          0
Exterior1st       0
Exterior2nd       0
MasVnrType      872
MasVnrArea        8
ExterQual         0
ExterCond         0
Foundation        0
Heating           0
HeatingQC         0
CentralAir        0
Electrical        1
dtype: int64

# Análisis de variables categóricas 


In [32]:
cat_cols = [
    'MasVnrType', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
    'Foundation', 'Heating', 'CentralAir', 'Electrical',
    'ExterQual', 'ExterCond', 'HeatingQC'
]

df[cat_cols].head()


,MasVnrType,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,Foundation,Heating,CentralAir,Electrical,ExterQual,ExterCond,HeatingQC
0,BrkFace,Gable,CompShg,VinylSd,VinylSd,PConc,GasA,Y,SBrkr,Gd,TA,Ex
1,NaN,Gable,CompShg,MetalSd,MetalSd,CBlock,GasA,Y,SBrkr,TA,TA,Ex
2,BrkFace,Gable,CompShg,VinylSd,VinylSd,PConc,GasA,Y,SBrkr,Gd,TA,Ex
3,NaN,Gable,CompShg,Wd Sdng,Wd Shng,BrkTil,GasA,Y,SBrkr,TA,TA,Gd
4,BrkFace,Gable,CompShg,VinylSd,VinylSd,PConc,GasA,Y,SBrkr,Gd,TA,Ex


In [33]:
df[cat_cols].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   MasVnrType   588 non-null    object
 1   RoofStyle    1460 non-null   object
 2   RoofMatl     1460 non-null   object
 3   Exterior1st  1460 non-null   object
 4   Exterior2nd  1460 non-null   object
 5   Foundation   1460 non-null   object
 6   Heating      1460 non-null   object
 7   CentralAir   1460 non-null   object
 8   Electrical   1459 non-null   object
 9   ExterQual    1460 non-null   object
 10  ExterCond    1460 non-null   object
 11  HeatingQC    1460 non-null   object
dtypes: object(12)
memory usage: 137.0+ KB


- Se observa que `MasVnrType`  tienen varios nulos y se asume que los NaNs en las columnas se deben a la ausencia de mampostería en la vivienda. Se determinó que para este caso una imputación por **'None'** para MasVnrType es la estrategia más adecuada, tratando el nulo como una ausencia intencional.



## Corregir MasVnrType 

In [38]:
print("Distribución de MasVnrType antes de imputación:")
print(df['MasVnrType'].value_counts(dropna=False))

Distribución de MasVnrType antes de imputación:
MasVnrType
None       872
BrkFace    445
Stone      128
BrkCmn      15
Name: count, dtype: int64


In [ ]:
#Imputar nulos de MasVnrType con 'None'
df['MasVnrType'] = df['MasVnrType'].fillna('None')
df.head()




Distribución de MasVnrType DESPUÉS de imputación:
MasVnrType
None       872
BrkFace    445
Stone      128
BrkCmn      15
Name: count, dtype: int64


In [ ]:
print("\nDistribución de MasVnrType después de imputación:")
print(df['MasVnrType'].value_counts(dropna=False))